In [76]:
# improt all necessary modules and library
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Embedding,LSTM,Dense

In [97]:
data =  pd.read_csv('medical_data.csv')
data.head()

,Patient_Problem,Disease,Prescription
0,"Constant fatigue and muscle weakness, struggli...",Chronic Fatigue Syndrome,"Cognitive behavioral therapy, graded exercise ..."
1,"Frequent severe migraines, sensitivity to ligh...",Migraine with Aura,"Prescription triptans, avoid triggers like bri..."
2,"Sudden weight gain and feeling cold, especiall...",Hypothyroidism,Levothyroxine to regulate thyroid hormone levels.
3,"High fever, sore throat, and swollen lymph nod...",Mononucleosis,"Rest and hydration, ibuprofen for pain."
4,"Excessive thirst and frequent urination, dry m...",Diabetes Mellitus,Insulin therapy and lifestyle changes.


Data preprocessing and preparation


In [78]:
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(data['Patient_Problem']);

sequences = tokenizer.texts_to_sequences(data['Patient_Problem'])

In [79]:
max_length = max(len(x) for x in sequences)

padded_sequences = pad_sequences(sequences,maxlen=max_length,padding='post');


label encoding and converting them into categorical

In [80]:
# encoding the labels
label_encoder_disease = LabelEncoder()
label_encoder_prescription = LabelEncoder()

disease_label = label_encoder_disease.fit_transform(data['Disease'])
prescription_label = label_encoder_prescription.fit_transform(data['Prescription'])

# converting label into categorical
disease_lebel_categorical = to_categorical(disease_label) 
prescription_label_categorical = to_categorical(prescription_label)


In [81]:
Y = np.hstack((disease_lebel_categorical, prescription_label_categorical))
Y.shape

(407, 566)

Model Building

In [82]:
input_layer = Input(shape=(max_length,))
embedding = Embedding(input_dim=5000,output_dim=64)(input_layer)
lstm_layer = LSTM(64)(embedding)

disease_output = Dense(len(label_encoder_disease.classes_),activation='softmax',name='disease_output')(lstm_layer)

prescription_output = Dense(len(label_encoder_prescription.classes_),activation='softmax',name='prescription_output')(lstm_layer)

Compiling the Model

In [83]:
model = Model(inputs=input_layer, outputs=[disease_output, prescription_output])

model.compile(
    loss={'disease_output': 'categorical_crossentropy', 
    'prescription_output': 'categorical_crossentropy'},
    optimizer='adam',
    metrics={'disease_output': ['accuracy'], 'prescription_output': ['accuracy']}
)

model.summary()

Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8       │ (None, 17)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_7         │ (None, 17, 64)    │    320,000 │ input_layer_8[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_13 (LSTM)      │ (None, 64)        │     33,024 │ embedding_7[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ disease_output      │ (None, 178)       │     11,570 │ lstm_13[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ prescription_output │ (None, 388)       │     25,220 │ lstm_13[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 389,814 (1.49 MB)

 Trainable params: 389,814 (1.49 MB)

 Non-trainable params: 0 (0.00 B)

In [88]:
model.fit(padded_sequences, {'disease_output': disease_lebel_categorical, 'prescription_output':
      prescription_label_categorical}, epochs=150, batch_size=32)

Epoch 1/150
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - disease_output_accuracy: 0.9553 - loss: 1.1294 - prescription_output_accuracy: 0.7984
Epoch 2/150
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - disease_output_accuracy: 0.9543 - loss: 1.0216 - prescription_output_accuracy: 0.8385
Epoch 3/150
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - disease_output_accuracy: 0.9545 - loss: 0.9887 - prescription_output_accuracy: 0.8489
Epoch 4/150
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - disease_output_accuracy: 0.9322 - loss: 1.0554 - prescription_output_accuracy: 0.8470
Epoch 5/150
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - disease_output_accuracy: 0.9608 - loss: 0.9775 - prescription_output_accuracy: 0.8716
Epoch 6/150
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - disease_output_accuracy: 0.9567 - loss: 0.9963 - prescription_output_accuracy: 0.8293
Epoch 7/150
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - disease_output_accuracy: 0.9652 - loss: 0.9429 - prescription_output_accuracy: 0.8702
Epoch 8/150
13/13 ━━

Making predictions

In [89]:
def make_prediction(patient_problem):
    # Preprocessing the input
    sequence = tokenizer.texts_to_sequences([patient_problem])
    padded_sequence = pad_sequences(sequence, maxlen=max_length, padding='post')
    
    # Making prediction
    prediction = model.predict(padded_sequence)
    
    # Decoding the prediction
    disease_index = np.argmax(prediction[0], axis=1)[0]
    prescription_index = np.argmax(prediction[1], axis=1)[0]
    
    disease_predicted = label_encoder_disease.inverse_transform([disease_index])[0]
    prescription_predicted = label_encoder_prescription.inverse_transform([prescription_index])[0]
    
    print(f"Predicted Disease: {disease_predicted}")
    print(f"Suggested Prescription: {prescription_predicted}")

    

In [1]:
patient_input = "headache, sweating "
make_prediction(patient_input)

NameError: name 'make_prediction' is not defined